[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RizanSM/zero_shot_llms_in_HIL_RL/blob/main/01_Code/02_Reacher_Env/02_Default_Environment/04_LLM_DIRECT_REACHER/03_Model_testing_LLM_D_Reacher_Ideal.ipynb)

In [ ]:
# Install the required libraries in your Google Colab environment
!pip install gymnasium mujoco stable-baselines3 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
# Import the necessary libraries
import gymnasium as gym
import numpy as np
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt

In [ ]:
from stable_baselines3 import PPO
from google.colab import drive
from google.colab import data_table

# Mount Google Drive
drive.mount('/content/drive')

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


Mounted at /content/drive


Average Episodic Reward

In [ ]:
def average_episodic_reward(test_trajectory_df):
    # Step 1: Calculate Cumulative Reward Per Episode
    cumulative_rewards = test_trajectory_df.groupby('Episode')['Reward'].sum().reset_index()
    cumulative_rewards.columns = ['episode', 'cumulative_reward']

    # Step 2: Calculate Variance and Standard Deviation of Cumulative Rewards
    variance = cumulative_rewards['cumulative_reward'].var()
    std_dev = cumulative_rewards['cumulative_reward'].std()
    final_reward = cumulative_rewards['cumulative_reward'].mean()

    # Step 4: Return the dataframe containing all results
    return cumulative_rewards, variance, std_dev, final_reward

COMBINING ALL THE DATA FRAMES

In [ ]:
def create_cummulative_reward(df1, df2, df3, df4, df5):
    """
    Create a data frame 'ideal_cummulative_reward' that contains:
      - 'episode' column (common across all data frames)
      - 'cumulative_reward_1' to 'cumulative_reward_5' columns from each data frame respectively,
        where each input data frame has columns 'episode' and 'cummulative_reward'.
      - 'mean_cumulative_reward' column containing the row-wise mean of the 5 cumulative rewards.

    Parameters:
      df1, df2, df3, df4, df5 (pd.DataFrame): Data frames with columns 'episode' and 'cummulative_reward'.

    Returns:
      pd.DataFrame: The merged and aggregated data frame.
    """
    # Rename the 'cummulative_reward' column in each data frame to a unique name.
    df1_renamed = df1.rename(columns={'cumulative_reward': 'cumulative_reward_1'})
    df2_renamed = df2.rename(columns={'cumulative_reward': 'cumulative_reward_2'})
    df3_renamed = df3.rename(columns={'cumulative_reward': 'cumulative_reward_3'})
    df4_renamed = df4.rename(columns={'cumulative_reward': 'cumulative_reward_4'})
    df5_renamed = df5.rename(columns={'cumulative_reward': 'cumulative_reward_5'})

    # Merge the data frames on the 'episode' column.
    merged_df = df1_renamed[['episode', 'cumulative_reward_1']].copy()
    merged_df = merged_df.merge(df2_renamed[['episode', 'cumulative_reward_2']], on='episode')
    merged_df = merged_df.merge(df3_renamed[['episode', 'cumulative_reward_3']], on='episode')
    merged_df = merged_df.merge(df4_renamed[['episode', 'cumulative_reward_4']], on='episode')
    merged_df = merged_df.merge(df5_renamed[['episode', 'cumulative_reward_5']], on='episode')

    # Compute the episode-wise mean of the cumulative rewards.
    reward_columns = [
        'cumulative_reward_1',
        'cumulative_reward_2',
        'cumulative_reward_3',
        'cumulative_reward_4',
        'cumulative_reward_5'
    ]
    merged_df['mean_cumulative_reward'] = merged_df[reward_columns].mean(axis=1)

    return merged_df

VARIANCE, STANDARD DEVIATION & FINAL REWARD

In [ ]:
def calculate_performance(variance_values, std_dev_values, final_reward_values):
    """
    Calculate performance metrics by computing the mean of each provided list.

    Parameters:
        variance_values (list of float): A list containing 5 variance values.
        std_dev_values (list of float): A list containing 5 standard deviation values.
        final_reward_values (list of float): A list containing 5 final reward values.

    Returns:
        tuple: A tuple containing three values:
            - mean_variance (float): Mean of the variance values.
            - mean_std_dev (float): Mean of the standard deviation values.
            - mean_final_reward (float): Mean of the final reward values.
    """
    mean_variance = sum(variance_values) / len(variance_values)
    mean_std_dev = sum(std_dev_values) / len(std_dev_values)
    mean_final_reward = sum(final_reward_values) / len(final_reward_values)

    return mean_variance, mean_std_dev, mean_final_reward


0. MODEL TESTING (LLM_D_Ideal)

In [ ]:
trajectory_df_1 = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/03_test_trajectories/4_llm_d/1_llm_d_reacher_df_1.pkl')      # Update directory location 1
trajectory_df_2 = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/03_test_trajectories/4_llm_d/2_llm_d_reacher_df_1.pkl')      # Update directory location 2
trajectory_df_3 = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/03_test_trajectories/4_llm_d/3_llm_d_reacher_df_1.pkl')      # Update directory location 3
trajectory_df_4 = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/03_test_trajectories/4_llm_d/4_llm_d_reacher_df_1.pkl')      # Update directory location 4
trajectory_df_5 = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/03_test_trajectories/4_llm_d/5_llm_d_reacher_df_1.pkl')      # Update directory location 5

TESTING THE MODEL BY ANALYZING THE DATA FRAME <br>



1. LLM feedback Direct Reacher Ideal data frame 1<br>
trajectory_df_1 ---> 1_llm_d_reacher_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_1)

Output hidden; open in https://colab.research.google.com to view.

A. Average Episodic Reward (trajectory_df_1)

In [ ]:
learning_curve_result_df_1, variance_1, std_dev_1, average_episodic_reward_1 = average_episodic_reward(trajectory_df_1)
print("Variance:", variance_1)
print("Standard Deviation:", std_dev_1)
print("Average Episodic Reward:", average_episodic_reward_1)

Variance: 2.8685851098945886e-29
Standard Deviation: 5.355917390974761e-15
Average Episodic Reward: -9.622934586843035


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_1)

,episode,cumulative_reward
0,1,-9.622935
1,2,-9.622935
2,3,-9.622935
3,4,-9.622935
4,5,-9.622935
...,...,...
95,96,-9.622935
96,97,-9.622935
97,98,-9.622935
98,99,-9.622935


2. LLM feedback Direct Reacher Ideal data frame 2<br>
trajectory_df_2 ---> 2_llm_d_reacher_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_2)

Output hidden; open in https://colab.research.google.com to view.

A. Average Episodic Reward (trajectory_df_2)

In [ ]:
learning_curve_result_df_2, variance_2, std_dev_2, average_episodic_reward_2 = average_episodic_reward(trajectory_df_2)
print("Variance:", variance_2)
print("Standard Deviation:", std_dev_2)
print("Average Episodic Reward:", average_episodic_reward_2)

Variance: 5.386565373024282e-28
Standard Deviation: 2.3208975360890628e-14
Average Episodic Reward: -12.938287254724509


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_2)

,episode,cumulative_reward
0,1,-12.938287
1,2,-12.938287
2,3,-12.938287
3,4,-12.938287
4,5,-12.938287
...,...,...
95,96,-12.938287
96,97,-12.938287
97,98,-12.938287
98,99,-12.938287


3. LLM feedback Direct Reacher Ideal data frame 3<br>
trajectory_df_3 ---> 3_llm_d_reacher_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_3)

Output hidden; open in https://colab.research.google.com to view.

A. Average Episodic Reward (trajectory_df_3)

In [ ]:
learning_curve_result_df_3, variance_3, std_dev_3, average_episodic_reward_3 = average_episodic_reward(trajectory_df_3)
print("Variance:", variance_3)
print("Standard Deviation:", std_dev_3)
print("Average Episodic Reward:", average_episodic_reward_3)

Variance: 7.968291971929412e-29
Standard Deviation: 8.926528984957934e-15
Average Episodic Reward: -7.976832655515965


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_3)

,episode,cumulative_reward
0,1,-7.976833
1,2,-7.976833
2,3,-7.976833
3,4,-7.976833
4,5,-7.976833
...,...,...
95,96,-7.976833
96,97,-7.976833
97,98,-7.976833
98,99,-7.976833


4. LLM feedback Direct Reacher Ideal data frame 4<br>
trajectory_df_4 ---> 4_llm_d_reacher_df


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_4)

Output hidden; open in https://colab.research.google.com to view.

A. Average Episodic Reward (trajectory_df_4)

In [ ]:
learning_curve_result_df_4, variance_4, std_dev_4, average_episodic_reward_4 = average_episodic_reward(trajectory_df_4)
print("Variance:", variance_4)
print("Standard Deviation:", std_dev_4)
print("Average Episodic Reward:", average_episodic_reward_4)

Variance: 0.0
Standard Deviation: 0.0
Average Episodic Reward: -15.377744874373091


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_4)

,episode,cumulative_reward
0,1,-15.377745
1,2,-15.377745
2,3,-15.377745
3,4,-15.377745
4,5,-15.377745
...,...,...
95,96,-15.377745
96,97,-15.377745
97,98,-15.377745
98,99,-15.377745


5. LLM feedback Direct Reacher Ideal data frame 5<br>
trajectory_df_5 ---> 5_llm_d_reacher_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_5)

Output hidden; open in https://colab.research.google.com to view.

A. Average Episodic Reward (trajectory_df_5)

In [ ]:
learning_curve_result_df_5, variance_5, std_dev_5, average_episodic_reward_5 = average_episodic_reward(trajectory_df_5)
print("Variance:", variance_5)
print("Standard Deviation:", std_dev_5)
print("Average Episodic Reward:", average_episodic_reward_5)

Variance: 0.0
Standard Deviation: 0.0
Average Episodic Reward: -12.272812026475847


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_5)

,episode,cumulative_reward
0,1,-12.272812
1,2,-12.272812
2,3,-12.272812
3,4,-12.272812
4,5,-12.272812
...,...,...
95,96,-12.272812
96,97,-12.272812
97,98,-12.272812
98,99,-12.272812


COMBINED DATA FRAMES - LLM-D-IDEAL

In [ ]:
cummulative_reward = create_cummulative_reward(
        learning_curve_result_df_1,
        learning_curve_result_df_2,
        learning_curve_result_df_3,
        learning_curve_result_df_4,
        learning_curve_result_df_5
    )

data_table.enable_dataframe_formatter()
data_table.DataTable(cummulative_reward)

,episode,cumulative_reward_1,cumulative_reward_2,cumulative_reward_3,cumulative_reward_4,cumulative_reward_5,mean_cumulative_reward
0,1,-9.622935,-12.938287,-7.976833,-15.377745,-12.272812,-11.637722
1,2,-9.622935,-12.938287,-7.976833,-15.377745,-12.272812,-11.637722
2,3,-9.622935,-12.938287,-7.976833,-15.377745,-12.272812,-11.637722
3,4,-9.622935,-12.938287,-7.976833,-15.377745,-12.272812,-11.637722
4,5,-9.622935,-12.938287,-7.976833,-15.377745,-12.272812,-11.637722
...,...,...,...,...,...,...,...
95,96,-9.622935,-12.938287,-7.976833,-15.377745,-12.272812,-11.637722
96,97,-9.622935,-12.938287,-7.976833,-15.377745,-12.272812,-11.637722
97,98,-9.622935,-12.938287,-7.976833,-15.377745,-12.272812,-11.637722
98,99,-9.622935,-12.938287,-7.976833,-15.377745,-12.272812,-11.637722


VARIANCE, STANDARD DEVIATION & FINAL REWARD : LLM-D-IDEAL

In [ ]:
 # Organize the values into lists
variance_list = [variance_1, variance_2, variance_3, variance_4, variance_5]
std_dev_list = [std_dev_1, std_dev_2, std_dev_3, std_dev_4, std_dev_5]
final_reward_list = [average_episodic_reward_1, average_episodic_reward_2, average_episodic_reward_3, average_episodic_reward_4, average_episodic_reward_5]

# Call the function to compute the mean performance values
mean_variance, mean_std_dev, mean_final_reward = calculate_performance(variance_list, std_dev_list, final_reward_list)

# Print the results
print("LLM-D-IDEAL Mean Variance:", mean_variance)
print("LLM-D-IDEAL  Mean Standard Deviation:", mean_std_dev)
print("LLM-D-IDEAL  Mean Average Episodic Reward:", mean_final_reward)


LLM-D-IDEAL Mean Variance: 1.2940506162413365e-28
LLM-D-IDEAL  Mean Standard Deviation: 7.498284347364666e-15
LLM-D-IDEAL  Mean Average Episodic Reward: -11.63772227958649
